In [1]:
import pandas as pd
import numpy as np
import os
import pickle
from sklearn.metrics.pairwise import cosine_similarity

In [4]:
os.listdir('../artifacts')

['tfidf_vectorizer.pkl',
 'collaborative_model.pkl',
 'ratings.pkl',
 'content_df.pkl',
 'books.pkl',
 'users.pkl',
 'feature_vectors.pkl',
 'collaborative_pivot_table.pkl',
 '.ipynb_checkpoints',
 'popular_books.pkl']

In [5]:
# --- Step 1: Load All Model Artifacts ---
# We need to load all the final artifacts from our previous models.

# Popularity Model Artifact
popular_df = pd.read_pickle('../artifacts/popular_books.pkl')
    
# Content-Based Model Artifacts
content_df = pd.read_pickle('../artifacts/content_df.pkl')
feature_vectors = pickle.load(open('../artifacts/feature_vectors.pkl', 'rb'))
    
# Collaborative Filtering Model Artifacts
collaborative_model = pickle.load(open('../artifacts/collaborative_model.pkl', 'rb'))
pivot_table = pd.read_pickle('../artifacts/collaborative_pivot_table.pkl')
    

In [7]:
popular_df.head()

,Book-Title,Book-Author,Image-URL-M,num_ratings,Avg-Book-Rating
0,Harry Potter and the Order of the Phoenix (Boo...,J. K. Rowling,http://images.amazon.com/images/P/043935806X.0...,334,6.785928
1,The Perfect Storm : A True Story of Men Agains...,Sebastian Junger,http://images.amazon.com/images/P/006101351X.0...,364,5.196429
2,The Hobbit : The Enchanting Prelude to The Lor...,J.R.R. TOLKIEN,http://images.amazon.com/images/P/0345339681.0...,281,5.007117
3,To Kill a Mockingbird,Harper Lee,http://images.amazon.com/images/P/0446310786.0...,389,4.920308
4,Harry Potter and the Chamber of Secrets (Book 2),J. K. Rowling,http://images.amazon.com/images/P/0439064872.0...,351,4.729345


In [9]:
content_df.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Avg-Book-Rating,Image-URL-M,num_ratings,tags
0,1551668300,16 Lighthouse Road,DebbieMacomber,2001,Mira,1.907692,http://images.amazon.com/images/P/1551668300.0...,65,16 lighthouse road debbiemacomber mira 2001
1,0451524934,1984,GeorgeOrwell,1990,SignetBook,4.614583,http://images.amazon.com/images/P/0451524934.0...,192,1984 georgeorwell signetbook 1990
2,0316666009,1st to Die: A Novel,JamesPatterson,2001,LittleBrownandCompany,3.830508,http://images.amazon.com/images/P/0316666009.0...,118,1st to die: a novel jamespatterson littlebrown...
3,0446610038,1st to Die: A Novel,JamesPatterson,2002,WarnerVision,3.498721,http://images.amazon.com/images/P/0446610038.0...,391,1st to die: a novel jamespatterson warnervisio...
4,0345303067,2010: Odyssey Two,ArthurC.Clarke,1984,DelReyBooks,2.721519,http://images.amazon.com/images/P/0345303067.0...,79,2010: odyssey two arthurc.clarke delreybooks 1984


In [12]:
feature_vectors

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 13004 stored elements and shape (2140, 3417)>

In [17]:
pivot_table.head()

User-ID,254,2276,2766,2977,3363,4017,4385,6251,6323,6543,...,271705,273979,274004,274061,274301,274308,275970,277427,277639,278418
Book-Title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4 Blondes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A Bend in the Road,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
collaborative_model

,n_neighbors,5
,radius,1.0
,algorithm,'brute'
,leaf_size,30
,metric,'cosine'
,p,2
,metric_params,None
,n_jobs,None


In [23]:
# --- Step 2: Recreate Individual Recommendation Functions ---
# We'll create functions to get recommendations from each model.

# 1. Content Based Recommendation Function
def get_content_based_recs(book_title, data, vectors, n_recs=10):
    """Gets content-based recommendations."""
    try:
        book_index = data[data['Book-Title'] == book_title].index[0]
        target_vector = vectors[book_index]
        similarity_scores = cosine_similarity(target_vector, vectors)[0]
        books_list = sorted(list(enumerate(similarity_scores)), reverse=True, key=lambda x: x[1])
        
        recommendations = []
        for i in books_list[1:n_recs+1]:
            recommendations.append(data.iloc[i[0]]['Book-Title'])
        return recommendations
    except (IndexError, KeyError):
        return []

In [19]:
# 2. Collaborative Based Recommendation Function
def get_collaborative_recs(book_title, pivot, model, n_recs=10):
    """Gets collaborative filtering recommendations."""
    try:
        book_index = np.where(pivot.index == book_title)[0][0]
        distances, indices = model.kneighbors(pivot.iloc[book_index, :].values.reshape(1, -1), n_neighbors=n_recs + 1)
        
        recommendations = []
        for i in range(1, len(distances.flatten())):
            recommendations.append(pivot.index[indices.flatten()[i]])
        return recommendations
    except (IndexError, KeyError):
        return []


In [20]:
# 3. Popularity Based Recommendation Function
def get_popularity_recs(popular_df, n_recs=10):
    """Gets popularity-based recommendations."""
    return popular_df['Book-Title'].head(n_recs).tolist()

In [27]:
# --- Step 3: Build the Hybrid Recommendation Function ---
# This function will combine the results from all models.

def get_hybrid_recommendations(book_title, n_recs=10):
    print(f"\n--- Generating Hybrid Recommendations for: '{book_title}' ---")
    
    # 1. Get recommendations from the collaborative model (most personalized)
    collab_recs = get_collaborative_recs(book_title, pivot_table, collaborative_model, n_recs)
    
    # 2. Get recommendations from the content-based model
    content_recs = get_content_based_recs(book_title, content_df, feature_vectors, n_recs)
    
    # 3. Get recommendations from the popularity model (fallback)
    popular_recs = get_popularity_recs(popular_df, n_recs)
    
    # --- Hybrid Strategy: Combine and De-duplicate ---
    # We will prioritize collaborative, then content, then popular.
    
    final_recommendations = []
    
    # Add collaborative recommendations first
    for rec in collab_recs:
        if rec not in final_recommendations and rec != book_title:
            final_recommendations.append(rec)
            
    # Add content-based recommendations if we still need more
    for rec in content_recs:
        if len(final_recommendations) < n_recs and rec not in final_recommendations and rec != book_title:
            final_recommendations.append(rec)
            
    # Add popular recommendations as a fallback if we still need more
    for rec in popular_recs:
        if len(final_recommendations) < n_recs and rec not in final_recommendations and rec != book_title:
            final_recommendations.append(rec)
            
    # Print the final list
    if not final_recommendations:
        print("Could not generate recommendations. Please try another book.")
        return

    print("\n--- Final Hybrid Recommendation List ---")
    for i, rec in enumerate(final_recommendations):
        print(f"{i+1}. {rec}")


In [28]:
# --- Example Usage ---
# Let's test our hybrid system with a few books.

# A book that is likely in all models
get_hybrid_recommendations("The Da Vinci Code")

# Another popular book
get_hybrid_recommendations("The Lovely Bones: A Novel")

# A book that might only be in the content or popularity model
get_hybrid_recommendations("The Hobbit")


--- Generating Hybrid Recommendations for: 'The Da Vinci Code' ---

--- Final Hybrid Recommendation List ---
1. Angels &amp; Demons
2. Touching Evil
3. Saving Faith
4. Middlesex: A Novel
5. The Blue Nowhere : A Novel
6. Sea Glass: A Novel
7. The Lovely Bones: A Novel
8. Last Man Standing
9. The Jester
10. Mystic River

--- Generating Hybrid Recommendations for: 'The Lovely Bones: A Novel' ---

--- Final Hybrid Recommendation List ---
1. Where the Heart Is (Oprah's Book Club (Paperback))
2. Good in Bed
3. The Book of Ruth (Oprah's Book Club (Paperback))
4. Life of Pi
5. Lucky : A Memoir
6. The Pilot's Wife : A Novel
7. Drowning Ruth (Oprah's Book Club)
8. The Nanny Diaries: A Novel
9. Nights in Rodanthe
10. Empire Falls

--- Generating Hybrid Recommendations for: 'The Hobbit' ---

--- Final Hybrid Recommendation List ---
1. Harry Potter and the Order of the Phoenix (Book 5)
2. The Perfect Storm : A True Story of Men Against the Sea
3. The Hobbit : The Enchanting Prelude to The Lord of 

In [30]:
get_hybrid_recommendations("Harry Potter")


--- Generating Hybrid Recommendations for: 'Harry Potter' ---

--- Final Hybrid Recommendation List ---
1. Harry Potter and the Order of the Phoenix (Book 5)
2. The Perfect Storm : A True Story of Men Against the Sea
3. The Hobbit : The Enchanting Prelude to The Lord of the Rings
4. To Kill a Mockingbird
5. Harry Potter and the Chamber of Secrets (Book 2)
6. The Da Vinci Code
7. The Catcher in the Rye
8. Harry Potter and the Sorcerer's Stone (Harry Potter (Paperback))
9. The Five People You Meet in Heaven
10. The Fellowship of the Ring (The Lord of the Rings, Part 1)
